In [125]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objs as go
from sklearn.metrics import adjusted_rand_score
import joblib


data2 = pd.read_csv('C:/Users/DELL/Desktop/OpenClass/Formation/Projet_005/SuiviClient2PourML.csv')


# K-means

In [126]:
# Conversion de la colonne 'Date_Premier_Commande' en datetime si ce n'est pas déjà fait
data2['Date_Premier_Commande'] = pd.to_datetime(data2['Date_Premier_Commande'])

# Filtrage des données pour T0
data_T0 = data2[data2['Date_Premier_Commande'] < '2018-01-01']


In [127]:
# Sélection des colonnes RFM pour le clustering
data_rfm = data_T0[['Récence', 'Fréquence', 'Montant']]

# Initialisation du StandardScaler
scaler = StandardScaler()

# Standardisation des données
data_rfm_scaled = scaler.fit_transform(data_rfm)


In [128]:
# Imputation
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
data_rfm_scaled_imputed = imputer.fit_transform(data_rfm_scaled)

# Assurez-vous d'utiliser la variable 'data_rfm_scaled_imputed' pour le KMeans
kmeans_T0 = KMeans(n_clusters=4, random_state=42)
kmeans_T0.fit(data_rfm_scaled_imputed)

# Ajout des étiquettes de cluster au DataFrame original
# Note : Assurez-vous que 'data2' contient le même nombre de lignes que 'data_rfm_scaled_imputed' après l'imputation
data_T0['Cluster'] = kmeans_T0.labels_


c:\Users\DELL\anaconda3\envs\2024\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\DELL\AppData\Local\Temp\ipykernel_9996\290074714.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [129]:
# Calcul des moyennes pour chaque cluster
cluster_summary = data_T0.select_dtypes(include=[np.number]).groupby('Cluster').mean()
print(cluster_summary)

         customer_zip_code_prefix  nbr_commande  Total_achat     Récence  \
Cluster                                                                    
0                    35890.077325      1.000000   133.398994  314.390658   
1                    36843.412426      1.000000   132.217408  487.135613   
2                    34719.066547      2.146283   282.014754  292.706235   
3                    41960.831723      1.020934  1135.226892  388.993559   

         Fréquence      Montant  
Cluster                          
0         1.000000   133.398994  
1         1.000000   132.217408  
2         2.146283   282.014754  
3         1.020934  1135.226892  


## Cluster

In [130]:
# Votre variable 'data_rfm_scaled_imputed' contient les données standardisées et imputées
# Votre DataFrame 'data2' contient maintenant une colonne 'Cluster' avec les labels de cluster
silhouette_avg = silhouette_score(data_rfm_scaled_imputed, data_T0['Cluster'])
print(f"Le score moyen de silhouette pour les clusters est : {silhouette_avg:.2f}")


Le score moyen de silhouette pour les clusters est : 0.51


In [131]:
def simuler_clustering(data_T0, colonne_date, nombre_de_periodes, n_clusters):
    # Convertir la colonne de date en datetime et trier le DataFrame par cette colonne
    data_T0[colonne_date] = pd.to_datetime(data_T0[colonne_date])
    data_T0 = data_T0.sort_values(by=colonne_date)

    # Trouver la date la plus récente dans le dataset
    date_max = data_T0[colonne_date].max()

    # Définir la durée d'une période en jours
    duree_periode = (date_max - data_T0[colonne_date].min()) / nombre_de_periodes

    # Initialiser la liste pour stocker les résultats de clustering
    resultats_clustering = []

    # Créer les périodes et effectuer le clustering pour chacune
    for i in range(nombre_de_periodes):
        # Calculer les limites de la période actuelle
        fin_periode = date_max - i * duree_periode
        debut_periode = date_max - (i + 1) * duree_periode

        # Sélectionner les données pour la période actuelle
        donnees_periode = data_T0[(data_T0[colonne_date] > debut_periode) & (data_T0[colonne_date] <= fin_periode)]

        # Imputer les valeurs manquantes
        imputer = SimpleImputer(strategy='mean')
        donnees_periode = donnees_periode.copy()  # Faire une copie pour éviter les avertissements
        donnees_periode[rfm_columns] = imputer.fit_transform(donnees_periode[rfm_columns])

        # Appliquer le clustering K-Means si assez de données
        if len(donnees_periode) >= n_clusters:
            kmeans_T0 = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            donnees_periode['Cluster'] = kmeans_T0.fit_predict(donnees_periode[rfm_columns])

            # Ajouter les résultats au tableau
            resultats_clustering.append({
                'debut_periode': debut_periode,
                'fin_periode': fin_periode,
                'donnees': donnees_periode,
                'etiquettes': kmeans_T0.labels_
            })

    return resultats_clustering

# Vos colonnes RFM pourraient avoir des noms différents
rfm_columns = ['Récence', 'Fréquence', 'Montant']


In [132]:

resultats_clustering = simuler_clustering(data_T0, 'Date_Premier_Commande', 4, 4)  # vos_donnees, 'Date_Premier_Commande', nombre_de_periodes, nombre_de_clusters)

# Palette de couleurs personnalisée
couleurs_clusters = {'0': 'red', '1': 'blue', '2': 'green', '3': 'orange'}

data_with_periods = pd.concat([r['donnees'] for r in resultats_clustering], ignore_index=True)

# Conversion de 'Date_Derniere_Commande' en période (mois) et en chaîne de caractères pour Plotly
data_T0['periode'] = pd.to_datetime(data_T0['Date_Premier_Commande']).dt.to_period('M').astype(str)

# Calcul de la taille de chaque cluster par période
cluster_sizes = data_T0.groupby(['periode', 'Cluster']).size().reset_index(name='taille')

# Pivoter les données pour obtenir une colonne par cluster
pivot_table = cluster_sizes.pivot(index='periode', columns='Cluster', values='taille').fillna(0)

# Réinitialisation de l'index pour utiliser 'periode' comme une colonne normale
pivot_table = pivot_table.reset_index()

# Préparation des données pour le graphique linéaire
# Plotly a besoin de les avoir en format long
data_for_plot = pd.melt(pivot_table, id_vars='periode', var_name='Cluster', value_name='Taille')

# Création du graphique linéaire avec Plotly
fig = px.line(data_for_plot, x='periode', y='Taille', color='Cluster', color_discrete_map=couleurs_clusters,
              title='Évolution de la taille des clusters au fil du temps',
              labels={'periode': 'Période', 'Taille': 'Taille du Cluster', 'Cluster': 'Cluster'})

# Affichage du graphique
fig.show()



C:\Users\DELL\AppData\Local\Temp\ipykernel_9996\1424652538.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DELL\AppData\Local\Temp\ipykernel_9996\1499428896.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [133]:
# Assurez-vous que 'Date_Derniere_Commande' est convertie en datetime si ce n'est pas déjà fait
data_T0['Date_Premier_Commande'] = pd.to_datetime(data_T0['Date_Premier_Commande'])

# Créez la colonne 'periode'
data_T0['periode'] = data_T0['Date_Premier_Commande'].dt.to_period("M").astype(str)

# Vérifiez que la colonne 'periode' existe
print(data_T0['periode'].head())

# Sélectionnez les données numériques et la colonne 'periode'
data_T0_numeric_with_period = data_T0.select_dtypes(include=[np.number])
data_T0_numeric_with_period['periode'] = data_T0['periode']

# Vérifiez que 'periode' est dans le nouveau DataFrame
print(data_T0_numeric_with_period.head())

# Maintenant, effectuez le groupement
centroids = data_T0_numeric_with_period.groupby(['periode', 'Cluster']).mean().reset_index()
print(centroids.head())


2     2017-03
3     2017-10
4     2017-11
8     2017-03
10    2017-07
Name: periode, dtype: object
    customer_zip_code_prefix  nbr_commande  Total_achat  Récence  Fréquence  \
2                      88115             1        86.22      541          1   
3                      66812             1        43.62      325          1   
4                      18040             1       196.89      292          1   
8                      25966             1       150.12      547          1   
10                     29400             1        29.00      411          1   

    Montant  Cluster  periode  
2     86.22        1  2017-03  
3     43.62        0  2017-10  
4    196.89        0  2017-11  
8    150.12        1  2017-03  
10    29.00        1  2017-07  
   periode  Cluster  customer_zip_code_prefix  nbr_commande  Total_achat  \
0  2016-09        1              69309.000000           1.0   136.230000   
1  2016-10        1              39108.114286           1.0   158.042393   
2  201

C:\Users\DELL\AppData\Local\Temp\ipykernel_9996\2027190930.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DELL\AppData\Local\Temp\ipykernel_9996\2027190930.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [134]:

# Préparation des données pour le graphique linéaire - prenons la moyenne de la 'Récence' pour cet exemple
centroids = centroids.pivot(index='periode', columns='Cluster', values='Récence').fillna(0)

# Réinitialisation de l'index pour utiliser 'periode' comme une colonne normale
centroids = centroids.reset_index()

# Conversion des périodes en chaîne pour la compatibilité avec Plotly
centroids['periode'] = centroids['periode'].astype(str)

# Préparation des données pour Plotly - format long
data_for_plot = pd.melt(centroids, id_vars='periode', var_name='Cluster', value_name='Centroïdité_Récence')

# Création du graphique linéaire avec Plotly
fig = px.line(data_for_plot, x='periode', y='Centroïdité_Récence', color='Cluster', color_discrete_map=couleurs_clusters,
              title='Évolution de la centroïdité des clusters pour la Récence au fil du temps',
              labels={'periode': 'Période', 'Centroïdité_Récence': 'Centroïdité de la Récence', 'Cluster': 'Cluster'})

# Affichage du graphique
fig.show()


In [135]:
# Calcul des centroïdes pour chaque cluster et chaque période
centroids = data_T0_numeric_with_period.groupby(['periode', 'Cluster']).mean().reset_index()

# Conversion des périodes en chaînes pour la compatibilité avec Plotly
centroids['periode'] = centroids['periode'].astype(str)

# Créer une figure Plotly
fig = go.Figure()

# Palette de couleurs pour les clusters
colors = {
    0: ['red', 'darkred', 'lightcoral'],
    1: ['blue', 'darkblue', 'lightblue'],
    2: ['green', 'darkgreen', 'lightgreen'],
    3: ['purple', 'indigo', 'plum']
}

# Attributs RFM à tracer
rfm_attributes = ['Récence', 'Fréquence', 'Montant']

# Ajouter les traces pour chaque centroïde de chaque cluster
for cluster in centroids['Cluster'].unique():
    cluster_data = centroids[centroids['Cluster'] == cluster]
    for i, attr in enumerate(rfm_attributes):
        fig.add_trace(go.Scatter(
            x=cluster_data['periode'],
            y=cluster_data[attr],
            mode='lines+markers',
            name=f'Cluster {cluster} - {attr}',
            line=dict(color=colors[cluster][i]),
            showlegend=True
        ))

# Mise à jour du layout du graphique pour ajouter les titres et légendes
fig.update_layout(
    title='Évolution des centroïdes des clusters pour la Récence, la Fréquence et le Montant au fil du temps',
    xaxis_title='Période',
    yaxis_title='Valeur du centroïde',
    legend_title='Attributs RFM par Cluster'
)

# Afficher le graphique
fig.show()


# Simulation Maintenance

## Période +X jours à définir

In [136]:
# Sélection des nouveaux clients après 30 jours pour T1
date_T1_cutoff = data_T0['Date_Premier_Commande'].max() + pd.Timedelta(days=38)  # Ici, on change la valeurs de days = x pour observer le score ARI
new_clients_T1 = data2[(data2['Date_Premier_Commande'] > data_T0['Date_Premier_Commande'].max()) &
                       (data2['Date_Premier_Commande'] <= date_T1_cutoff)]

# Combinaison des anciens et des nouveaux clients pour T1
data_T1 = pd.concat([data_T0, new_clients_T1])

# Ré-entrainement du modèle avec data_T1
# Sélection des colonnes RFM pour le clustering
data_rfm = data_T1[['Récence', 'Fréquence', 'Montant']]

# Initialisation du StandardScaler
scaler = StandardScaler()

# Standardisation des données
data_rfm_scaled = scaler.fit_transform(data_rfm)

# Imputation
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
data_rfm_scaled_imputed = imputer.fit_transform(data_rfm_scaled)

# Assurez-vous d'utiliser la variable 'data_rfm_scaled_imputed' pour le KMeans
kmeans_T1 = KMeans(n_clusters=4, random_state=42)
kmeans_T1.fit(data_rfm_scaled_imputed)

# Ajout des étiquettes de cluster au DataFrame original
# Note : Assurez-vous que 'data2' contient le même nombre de lignes que 'data_rfm_scaled_imputed' après l'imputation
data_T1['Cluster'] = kmeans_T1.labels_

# Après avoir formé le modèle T1 et ajouté les étiquettes de cluster à data_T1
labels_T1_real = data_T1['Cluster'].values

cluster_summary = data_T1.select_dtypes(include=[np.number]).groupby('Cluster').mean()
print(cluster_summary)

c:\Users\DELL\anaconda3\envs\2024\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



         customer_zip_code_prefix  nbr_commande  Total_achat     Récence  \
Cluster                                                                    
0                    35776.052355      1.000000   130.474708  282.593930   
1                    41392.699701      1.015522  1053.576603  362.660896   
2                    34543.051255      2.134937   283.985429  276.824268   
3                    36741.683321      1.000000   129.988015  472.575691   

         Fréquence      Montant  
Cluster                          
0         1.000000   130.474708  
1         1.015522  1053.576603  
2         2.134937   283.985429  
3         1.000000   129.988015  


# ARI Contrôle 

## Mise en place

In [137]:
# Ajuster le scaler sur les données T0
scaler = StandardScaler()
scaler.fit(data_T0[['Récence', 'Fréquence', 'Montant']])

# Sauvegarder le scaler pour une utilisation ultérieure
# Vous pouvez utiliser un package de sérialisation comme joblib ou pickle pour cela
import joblib
joblib.dump(scaler, 'scaler_T0.joblib')
joblib.dump(kmeans_T0, 'kmeans_T0.joblib')

joblib.dump(scaler, 'scaler_T1.joblib')
joblib.dump(kmeans_T1, 'kmeans_T1.joblib')

# Pour utiliser le scaler plus tard, il suffit de le charger
scaler = joblib.load('scaler_T0.joblib')
scaler = joblib.load('scaler_T1.joblib')

# Maintenant, vous pouvez transformer vos données de prédiction
data_predict_scaled = scaler.transform(data_predict_rfm)


In [138]:
# Ajuster l'imputer sur les données T0
imputer = SimpleImputer(strategy='mean')
imputer.fit(data_T0[['Récence', 'Fréquence', 'Montant']])
imputer.fit(data_T1[['Récence', 'Fréquence', 'Montant']])

# Sauvegarder l'imputer pour une utilisation ultérieure
joblib.dump(imputer, 'imputer_T0.joblib')
joblib.dump(imputer, 'imputer_T1.joblib')

# Pour utiliser l'imputer plus tard, il suffit de le charger
imputer = joblib.load('imputer_T0.joblib')
imputer = joblib.load('imputer_T1.joblib')

# Maintenant, vous pouvez imputer vos données de prédiction
data_predict_scaled_imputed = imputer.transform(data_predict_scaled)


c:\Users\DELL\anaconda3\envs\2024\Lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but SimpleImputer was fitted with feature names



## Résultat

In [139]:
# Charger les modèles et les scalers
scaler_T0 = joblib.load('scaler_T0.joblib')
kmeans_T0 = joblib.load('kmeans_T0.joblib')
kmeans_T1 = joblib.load('kmeans_T1.joblib')

# Assurer que les données RFM sont prêtes pour T1
data_T1_rfm = data_T1[['Récence', 'Fréquence', 'Montant']]

# Standardisation des données pour T1
data_T1_scaled = scaler_T0.transform(data_T1_rfm)

# Prédiction des clusters pour T1 avec le modèle de T0
labels_T1_pred_by_T0 = kmeans_T0.predict(data_T1_scaled)

# Obtenir les labels réels de T1
labels_T1_real = kmeans_T1.labels_

# Calcul de l'ARI
ari_score_T1 = adjusted_rand_score(labels_T1_real, labels_T1_pred_by_T0)
print("Adjusted Rand Index :", ari_score_T1)



Adjusted Rand Index : 0.7955524366678113


C'est à partir du jour 37 que l'ARI passe en-dessous de 0.8. Nous devrions donc revoir le modèle au plus tard tout les 37jours et si 